In [ ]:
import h5py
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
                .master("spark://192.168.2.34:7077")\
                .appName("Song Analysis")\
                .config("spark.hadoop.fs.defaultFS", "hdfs://192.168.2.34:9000") \
                .config("spark.hadoop.dfs.replication", "1") \
                .getOrCreate()


In [ ]:
path = "hdfs://192.168.2.34:9000/user/ubuntu/millionsongs/MillionSongSubset/A/A/A/TRAAAAW128F429D538.h5"
with h5py.File(path, "r") as hf:
    # Read the data from the relevant groups
    duration = hf["/analysis/songs"][0]["duration"][()]
    genre = hf["/metadata/songs"][0]["genre"][()].decode("utf-8")
    artist_name = hf["/metadata/songs"][0]["artist_name"][()].decode("utf-8")
    title = hf["/metadata/songs"][0]["title"][()].decode("utf-8")

# Print the data
print("Duration:", duration)
print("Genre:", genre)
print("Artist:", artist_name)
print("Title:", title)

In [ ]:
def read_h5_file(file_path):
    with h5py.File(file_path, "r") as hf:
        # Extract the data from the HDF5 file
        duration = hf["/duration"][()]
        genre = hf["/genre"][()].decode("utf-8")
        artist_name = hf["/artist_name"][()].decode("utf-8")
        title = hf["/title"][()].decode("utf-8")
    
    # Convert the data to a DataFrame
    data = [(duration, genre, artist_name, title)]
    return spark_session.createDataFrame(data, schema=["duration", "genre", "artist_name", "title"])

In [10]:
spark_session.stop()